# Unit 8.3 Questions 10 - 13

In [3]:
import sqlite3 as sq3

In [4]:
db_filename = 'sqlite_db_pythonsqlite.db'
try:
   connection = sq3.connect(db_filename)
   print(f"Database {db_filename} connected with SQLite3 ver {sq3.version}")
except sq3.Error as err:
    print("Couldn't connect to db file {db_filename}")
    print(err)

cur = connection.cursor()
print(cur)

Database sqlite_db_pythonsqlite.db connected with SQLite3 ver 2.6.0


/tmp/ipykernel_2102528/761827085.py:4: DeprecationWarning: version is deprecated and will be removed in Python 3.14
  print(f"Database {db_filename} connected with SQLite3 ver {sq3.version}")


In [5]:
# check basic functionality with a simple query
query_1 = 'select * from members limit 4'
cur.execute(query_1)
rows = cur.fetchall()
for row in rows:
    print(row)


(0, 'GUEST', 'GUEST', 'GUEST', 0, '(000) 000-0000', '', '2012-07-01 00:00:00')
(1, 'Smith', 'Darren', '8 Bloomsbury Close, Boston', 4321, '555-555-5555', '', '2012-07-02 12:02:05')
(2, 'Smith', 'Tracy', '8 Bloomsbury Close, New York', 4321, '555-555-5555', '', '2012-07-02 12:08:23')
(3, 'Rownam', 'Tim', '23 Highway Way, Boston', 23423, '(844) 693-0723', '', '2012-07-03 09:32:15')


In [30]:
# Question 10

# place-holder for now
query_10 = """   
WITH info_with_costs AS (
   SELECT 
        fac.name AS name,  
		fac.facid AS facid,
        bk.slots * (
            CASE 
                WHEN bk.memid = 0 THEN fac.guestcost 
                ELSE fac.membercost 
            END
        ) AS cost    -- from business POV, is revenue from each facility
   FROM Bookings AS bk
   LEFT JOIN Facilities AS fac ON fac.facid = bk.facid
   LEFT JOIN Members AS mem ON mem.memid = bk.memid
)

SELECT 
	facid,
    name AS facility,
    SUM(cost) as revenue
FROM info_with_costs
GROUP BY facid
HAVING revenue < 1000

ORDER BY cost DESC;
"""



cur.execute(query_10)
rows = cur.fetchall()
for row in rows:
    print(row)


(8, 'Pool Table', 270)
(7, 'Snooker Table', 240)
(3, 'Table Tennis', 180)


In [25]:
# Question 11: members and recommenders
# Listing all members, or just those who have a recommender?  
# I'll list just the ones with recommendations, using inner join.
# To list all, use left join.  b

query_11 = """
SELECT concat(mem.surname, ", ", mem.firstname), concat(rec.surname, ", ", rec.firstname)
FROM Members AS mem 
INNER JOIN Members AS rec  
   ON mem.recommendedby = rec.memid 
ORDER BY mem.surname, mem.firstname;
"""

cur.execute(query_11)
rows = cur.fetchall()
for row in rows:
    print(row)



('Bader, Florence', 'Stibbons, Ponder')
('Baker, Anne', 'Stibbons, Ponder')
('Baker, Timothy', 'Farrell, Jemima')
('Boothe, Tim', 'Rownam, Tim')
('Butters, Gerald', 'Smith, Darren')
('Coplin, Joan', 'Baker, Timothy')
('Crumpet, Erica', 'Smith, Tracy')
('Dare, Nancy', 'Joplette, Janice')
('Genting, Matthew', 'Butters, Gerald')
('Hunt, John', 'Purview, Millicent')
('Jones, David', 'Joplette, Janice')
('Jones, Douglas', 'Jones, David')
('Joplette, Janice', 'Smith, Darren')
('Mackenzie, Anna', 'Smith, Darren')
('Owen, Charles', 'Smith, Darren')
('Pinker, David', 'Farrell, Jemima')
('Purview, Millicent', 'Smith, Tracy')
('Rumney, Henrietta', 'Genting, Matthew')
('Sarwin, Ramnaresh', 'Bader, Florence')
('Smith, Jack', 'Smith, Darren')
('Stibbons, Ponder', 'Tracy, Burton')
('Worthington-Smyth, Henry', 'Smith, Tracy')


In [22]:
type(rows), type(rows[2])

(list, tuple)

In [27]:
# Nicer formatting 

fmt11 = "%-22s  %-22s"
print(fmt11 %  ("Member", "Recommender"))
for row in rows:
    print(fmt11 % (row[0], row[1]))

Member                  Recommender           
Bader, Florence         Stibbons, Ponder      
Baker, Anne             Stibbons, Ponder      
Baker, Timothy          Farrell, Jemima       
Boothe, Tim             Rownam, Tim           
Butters, Gerald         Smith, Darren         
Coplin, Joan            Baker, Timothy        
Crumpet, Erica          Smith, Tracy          
Dare, Nancy             Joplette, Janice      
Genting, Matthew        Butters, Gerald       
Hunt, John              Purview, Millicent    
Jones, David            Joplette, Janice      
Jones, Douglas          Jones, David          
Joplette, Janice        Smith, Darren         
Mackenzie, Anna         Smith, Darren         
Owen, Charles           Smith, Darren         
Pinker, David           Farrell, Jemima       
Purview, Millicent      Smith, Tracy          
Rumney, Henrietta       Genting, Matthew      
Sarwin, Ramnaresh       Bader, Florence       
Smith, Jack             Smith, Darren         
Stibbons, Pon

In [16]:
# Question 12: facility usage by members not guests
# Unclear exactly what is being asked for.
# Taking this to mean a list of all facilities, each with the total number of slots used by members, not counting guests.

query_12 = """
SELECT 
	fac.facid, fac.name, SUM(bk.slots) AS usage
FROM Bookings AS bk
   JOIN Facilities AS fac ON fac.facid = bk.facid
   JOIN Members AS mem ON mem.memid = bk.memid
WHERE mem.memid > 0 
GROUP BY fac.facid
ORDER BY usage DESC;
"""


cur.execute(query_12)
rows = cur.fetchall()
for row in rows:
    print(row)



(2, 'Badminton Court', 1086)
(0, 'Tennis Court 1', 957)
(4, 'Massage Room 1', 884)
(1, 'Tennis Court 2', 882)
(7, 'Snooker Table', 860)
(8, 'Pool Table', 856)
(3, 'Table Tennis', 794)
(6, 'Squash Court', 418)
(5, 'Massage Room 2', 54)


In [18]:
# Question 12-Modified: ask about only guest usage instead of only member usage (just curious)
# Only one change:  replace <> with = in the WHERE
#
# For total usage, guests and members, could remove the WHERE but 
# quicker to put memid >= 0, which can be quickly changed back to the other cases.

query_12_guests = """
SELECT 
	fac.facid, fac.name, SUM(bk.slots) AS usage
FROM Bookings AS bk
   JOIN Facilities AS fac ON fac.facid = bk.facid
   JOIN Members AS mem ON mem.memid = bk.memid
WHERE mem.memid = 0 
GROUP BY fac.facid
ORDER BY usage DESC;
"""


cur.execute(query_12_guests)
rows = cur.fetchall()
for row in rows:
    print(row)



(6, 'Squash Court', 686)
(4, 'Massage Room 1', 520)
(1, 'Tennis Court 2', 396)
(0, 'Tennis Court 1', 363)
(5, 'Massage Room 2', 174)
(2, 'Badminton Court', 123)
(8, 'Pool Table', 54)
(7, 'Snooker Table', 48)
(3, 'Table Tennis', 36)


* Interesting how Squash is tops with guests, but next to bottom in the members-only list. 
* Tennis and massage are popular with everyone. 
* Snooker has roughly 16x the guest activity compared to members.

In [21]:
# Question 13: monthly facility usage

query_13 = """
WITH withmonths AS (
   SELECT  
		substr('JanFebMarAprMayJunJulAugSepOctNovDec', 
		      1 + 3*strftime('%m',starttime),
		      -3)  AS month,
		fac.facid AS facid, 
		fac.name  AS facname,		
		bk.memid AS memid, 
		bk.slots AS slots
   FROM Bookings      AS bk
      JOIN Facilities AS fac  ON fac.facid = bk.facid
      JOIN Members    AS mem  ON mem.memid = bk.memid
   WHERE bk.memid > 0    -- don't count guests, only members
)

SELECT month, sum(slots) as total_slots    -- facid, facname, SUM(slots)
FROM withmonths
GROUP BY month
ORDER BY month;
"""


cur.execute(query_13)
rows = cur.fetchall()
for row in rows:
    print(row)



('Aug', 2531)
('Jul', 1061)
('Sep', 3199)


In [31]:
# END OF QUESTIONS - Close connection

# connection.commit()   # ...if writing to db, but we're not, so don't commit.
connection.close()
